In [90]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import urllib.request
import re
import cv2
import matplotlib.pyplot as plt
import os 

In [107]:
def clean_links(links: list): 
    new = []
    for i in range(len(links)): 
        if "m'{sid:,cturl:" in  links[i] or "m{&quot;sid&quot;:&quot;&quot;" in  links[i]:
            pass
        else: 
            new.append(links[i])
    # remove duplicate 
    return np.unique(new)



def download_image(image_name :  str ):
    # https://www.bing.com/images/search?q=dog&FORM=HDRSC3
    array_links = []
    for i in range(100):
        url = requests.get(f'https://www.bing.com/images/search?q={image_name}&form=HDRSC3&first={i}')
        soup = BeautifulSoup(url.content, 'lxml')

        get_image = soup.find_all(name = "div", attrs={"class" : "imgpt"})
            # print(url.status_code)
        if get_image: 
            # print(get_image)
            for attributes in str(get_image).split("src"):
                for i in attributes.split(" "): 
                    if "https://tse2.mm.bin" in i: 
                        url = i
                        url = url.replace(" ", "")
                        array_links.append(url.replace("=", "").replace('"', '').replace(" ", ""))
    return clean_links(array_links)
        #         for  atr in attributes:
        #             for link in str(atr).split(" "):
        #                 if "src" in link: 
        #                     url = link 
        #                     clean_url = re.sub(r'\s+', '', url)
        #                     clean_url = clean_url.replace("src", "")
        #                     clean_url = clean_url.replace("=", "")
        #                     clean_url = re.sub(r'"', '', clean_url)
        #                     array_links.append(clean_url)
        # else: 
        #     print("there is no img detected")
        # print("=========================================")                    
    # print(len(array_links))
    # for i in range(len(array_links)):
    #     if os.path.exists(f'./{image_name}'):
    #         full_path = f'./{image_name}/{image_name}.jpg'
    #         urllib.request.urlretrieve(array_links[i], full_path)
    #     else:
    #         os.mkdir(f"./{image_name}")
    #         full_path = f'./{image_name}/{image_name}{i + 1}.jpg'
    #         urllib.request.urlretrieve(array_links[i], full_path)
    
    # print("succesfully get " + str(len(array_links)) + " images")   
     




In [108]:
# url_image 
array_links = download_image(image_name="cat")
array_links

array(['https://tse2.mm.bing.net/th/id/ODF.1o0VEMY44vl8mbrZx0rT3A?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/ODF.5T1ZwMN3oTpGvoxrku4ZgQ?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/ODF.5lsl1crnf2xXBc4Fi8l3Jw?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/ODF.JXnFlcwaAvZHxw1uqd_ppQ?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/ODF._Bdx2LWvaVwh7BUB20le4w?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/ODF.bjMrY7920RJ0Dt4zRGIA6Q?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/ODF.qdU6pXmO5jYaUcvYTmBUmg?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/OIGP.nd8Q.pDNQ7qlOEobLNgP?w233&amp;h233&amp;c6&amp;o5&amp;pid1.7',
       'https://tse2.mm.bing.net/th/id/OIP.-0N0TQKZB1gOyfRKT5ioEgHaHa?w62&amp;h62&amp;c1&amp;rs1&amp;ql